In [1]:
#!pip install pygame

In [5]:
from math import *


class button():
    def __init__(self, color, x, y, width, height, text=''):
        self.color = color
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.text = text

    def draw(self, window):
        #Call this method to draw the button on the screen        
        pygame.draw.rect(window, self.color, (self.x, self.y, self.width, self.height), 0)

        if self.text != '':
            font = pygame.font.SysFont('Corbel', 40)
            text = font.render(self.text, 1, (0, 0, 0))
            window.blit(text, (
            self.x + (self.width / 2 - text.get_width() / 2), self.y + (self.height / 2 - text.get_height() / 2)))

    def isOver(self, pos):
        #Pos is the mouse position or a tuple of (x,y) coordinates
        if pos[0] > self.x and pos[0] < self.x + self.width:
            if pos[1] > self.y and pos[1] < self.y + self.height:
                return True

        return False


class disp_number():
    def __init__(self, color, x, y, width, height, number=0):
        self.color = color
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.number = number

    def draw(self, window):
        if self.number != 0:
            font = pygame.font.SysFont('Corbel', 40)
            text = font.render(str(self.number), 1, (0, 0, 0))
            window.blit(text, (
            self.x + (self.width / 2 - text.get_width() / 2), self.y + (self.height / 2 - text.get_height() / 2)))


class polygon():
    def __init__(self, color, vertex_count, radius, position):
        self.color = color
        self.vertex_count = vertex_count
        self.radius = radius
        self.position = position

    def draw_poly(self, window):
        #Call this method to draw the polygon on the screen        
        n, r = self.vertex_count, self.radius
        x, y = self.position
        color = self.color
        point_coords = [
            (x + r * cos(2 * pi * i / n), y + r * sin(2 * pi * i / n))
            for i in range(n)
        ]
        pygame.draw.polygon(window, color, point_coords)
        for point_coord in point_coords:
            pygame.draw.circle(window, (0,0,0), point_coord, 5)
        

class gameStats():
    score = 0
    lives = 5
    # The probabilities of polygons with edges 3,4,5,6,7,8,9
    probs = [1 / 7] * 7
    
    
import csv

# open the file in the write mode
gamelog = open('gamelog.csv', 'w', newline='')

# create the csv writer
writer = csv.writer(gamelog)

import pygame
import random
import textwrap

pygame.init()

gamerID = random.randrange(0, 10000000, 1)
gameID = random.randrange(0, 10000000, 1)
                    
#info = pygame.display.Info()
#print(info.current_w)
#print(info.current_h)

# To be implemented:
#screen = pygame.display.set_mode((info.current_w,info.current_h-50))

# Initiate game window
screen = pygame.display.set_mode((1920,1080))
pygame.display.set_caption("Game of shapes")

# colors
white = (255, 255, 255)
red = (255, 102, 102)
green = (102,255,102)
blue = (102,102,255)
yellow = (255,255,102)
teal = (102,255,255)
orange = (255,178,102)
pink = (255,102,178)
purple = (178,102,255)

# stores the width of the
# screen into a variable
width = screen.get_width()

# stores the height of the
# screen into a variable
height = screen.get_height()

size = 75

# buttons with numbers on them
s_minus = button((0, 200, 0), width / 2 - 4 * size / 2, height / 2 - size / 2, size, size, '-')
s_plus = button((0, 200, 0), width / 2 + 2 * size / 2, height / 2 - size / 2, size, size, '+')
s_enter = button((0, 200, 0), width / 2 - 2 * size / 2, height / 2 + size / 2 + 10, size * 2, size, 'Enter')

s_start = button((0,200,0),width/2-2*size/2,height/2-size/2,size*2,size, 'Start')

s_restart = button((0,200,0),width/2-2*size/2,height/2-size/2,size*2,size, 'Restart')

buttons = [s_minus, s_plus]#, s_enter]


#Possible polygon edges
polygons = [3, 4, 5, 6, 7, 8, 9]

# Selects how many edges the next polygon has
def nextPolygon():
    return random.choices(polygons, gameStats.probs)[0]

# Decrease probability of polygon appearing
def decreaseProbs(edge, correct=True):
    #when correctly guessed, then the probability of that polygon appearing again is decreased
    if correct:
        gameStats.probs[edge - 3] *= 0.8
        return [float(i) / sum(gameStats.probs) for i in gameStats.probs]

#---Initiating first polygon
edges = nextPolygon()
y = random.randrange(100, height - 100, 1)
x = random.randrange(100, width - 100, 1)
j = 0
#print(int(height / 2 - 300), int(height / 2 + 300), int(width / 2 - 300), int(width / 2 + 300))
while j == 0:
    if ((x in range(int(width / 2 - 200), int(width / 2 + 200))) & (
            y in range(int(height / 2 - 200), int(height / 2 + 200)))):
        y = random.randrange(100, height - 100, 1)
        x = random.randrange(100, width - 100, 1)
    else:
        j = 1

# First polygon
THEpolygon = polygon(white, edges, 100, (x, y))
#---Initiating first polygon done

# Default number
nr = disp_number((0, 0, 0), 0, 0, width, height, '6')

score_nr = disp_number((0, 0, 0), 0, -80, width, height, '0')

score_txt = disp_number((0, 0, 0), 0, -120, width, height, 'Score')

end_txt = disp_number((0, 0, 0), 0, -180, width, height, 'Out of lives!')

end_score_txt = disp_number((0, 0, 0), 0, -80, width, height, 'Score')

end_score_nr = disp_number((0, 0, 0), 0, -40, width, height, '0')

end_restart = disp_number((0, 0, 0), 0, 40, width, height, 'Press Enter to try again')

lives_nr = disp_number((0, 0, 0), 0, 120, width, height, '5')

lives_txt = disp_number((0, 0, 0), 0, 80, width, height, 'Lives')

#Polygon possible parameters
minr = 50
maxr = 150
colors = [white,red,green,blue,yellow,teal,orange,pink,purple]

def random_polygon():
    global THEpolygon
    y = random.randrange(100, height - 100, 1)
    x = random.randrange(100, width - 100, 1)
    i = 0
    while i == 0:
        #if the polygon would end up inside the middle circle, calculate new coordinates
        if ((x in range(int(width / 2 - (200+maxr)), int(width / 2 + (200+maxr)))) & (
                y in range(int(height / 2 - (200+maxr)), int(height / 2 + (200+maxr))))):
            y = random.randrange(100, height - 100, 1)
            x = random.randrange(100, width - 100, 1)
        #if the polygon would end up on the border, calculate new coordinates
        elif (x in range(0, maxr)) or (x in range(width-maxr, width)) or (y in range(0,maxr)) or (y in range(height-maxr, height)):
            y = random.randrange(100, height - 100, 1)
            x = random.randrange(100, width - 100, 1)
        else:
            THEpolygon.vertex_count = edges
            THEpolygon.position = (x, y)
            THEpolygon.radius = random.randint(minr, maxr)
            THEpolygon.color = random.choice(colors)
            i = 1
            print(x, y)


control = 1
def MOUSEoutOFcircle():
    global endGame, control
    pos = pygame.mouse.get_pos()
    x = pos[0]
    y = pos[1]
    sqx = (x - width / 2) ** 2
    sqy = (y - height / 2) ** 2
    if sqrt(sqx + sqy) > 180:
        screen.fill((250, 160, 160))
        if control - 1 == 0:
            gameStats.lives -= 1
            lives_nr.number = gameStats.lives
            if gameStats.lives <= 0:
                endGame = True
            control = 0
    else:
        screen.fill((224, 224, 224))
        control = 1
        
        
def keyPressed():  
    global endGame
    #Kui on klahvivajutus
    if event.type == pygame.KEYDOWN:
        global edges
        #vasakule võtab ühe numbri maha
        if event.key == pygame.K_LEFT:
            nr.number = int(nr.number) - 1
        #paremale paneb ühe numbri juurde
        elif event.key == pygame.K_RIGHT:
            nr.number = int(nr.number) + 1
        # enter või space sisestab vastuse
        elif event.key == pygame.K_SPACE or event.key == pygame.K_RETURN:
            seconds=(pygame.time.get_ticks()-start_ticks)/1000
            # write a row to the csv file
            row = []
            row.append(gamerID)
            row.append(gameID)
            row.append(nr.number)
            row.append(THEpolygon.vertex_count)
            row.append(THEpolygon.position[0])
            row.append(THEpolygon.position[1])
            row.append(THEpolygon.color[0])
            row.append(THEpolygon.color[1])
            row.append(THEpolygon.color[2])
            row.append(THEpolygon.radius)
            row.append(seconds)
            writer.writerow(row)
            
            #kui on õige vastus
            if int(nr.number) == edges:
                gameStats.score += edges
                score_nr.number = gameStats.score
                #print("Skoorisid!", score)
                gameStats.probs = decreaseProbs(edges)
                #print("Uued tõenäosused",probs)
                edges = nextPolygon()
                random_polygon()  # Example how to move polygon on click.
                
            #kui on vale vastus
            else:
                gameStats.lives -= 1
                lives_nr.number = gameStats.lives
                if gameStats.lives <= 0:
                    endGame = True
                edges = nextPolygon()
                #print("Ei skoorinud, tõenäosused jäävad samaks:",probs)
                random_polygon()
                
        else: #Not necessary but helpful for debugging
            print("vajutati nuppu, mida pole mapitud")
            print(pygame.key.name(event.key))

#I (Sille) put this in a function for clarity
def drawGameScreen():
    screen.fill((224, 224, 224))  # Background color
    MOUSEoutOFcircle()  # Check if mouse is in circle
    pygame.draw.circle(screen, (250, 250, 250), (width / 2, height / 2),
                       180)  # Middle circle (display, color, location, radius)
    for button in buttons:  # Draw the buttons
        button.draw(screen)
    THEpolygon.draw_poly(screen)  # Draw the polygon
    nr.draw(screen)
    
    score_txt.draw(screen) # Display score
    score_nr.draw(screen)
    
    lives_txt.draw(screen) # Display score
    lives_nr.draw(screen)

    pygame.display.update()  # Update the display
    
        
intro = True
running = False
endGame = False
while intro:
    screen.fill((224, 224, 224))
    font = pygame.font.SysFont('Corbel', 30)
    intro_message = "Welcome to the game of shapes! The goal of the game is to guess \
how many edges does the shape on your screen have, while only looking at the middle \
of the screen. If you guess correctly, you will gain as many points as the number of edges. \
If you guess wrong or look away from the middle for too long, you lose 1 life. The game ends after you lose your last life."
    #text = font.render(intro_message, 1, (0,0,0))
    wraplen = width/14
    txtX, txtY = 20, 20
    wrap_text = textwrap.TextWrapper(width=wraplen)
    for i in wrap_text.wrap(text = intro_message):
        txtY = txtY + 40
        Mtxt = font.render(f"{i}", True, (0, 0, 0))
        screen.blit(Mtxt, (txtX, txtY))
    txtY = height/1.4
    instruction_msg = "Your first task is to install and run GazePointer. After that, \
calibrate the vision there and choose the option to control the cursor with your eyes. \
From this point forward, you don't need to use your mouse. Use the left and right arrow \
keys to change the guessed number of edges and press Space or Enter to confirm your guess. \
When you are done reading, look at the start button in the middle and press Enter. Good luck!"
    for i in wrap_text.wrap(text = instruction_msg):
        txtY = txtY + 40
        Mtxt = font.render(f"{i}", True, (0, 0, 0))
        screen.blit(Mtxt, (txtX, txtY))
    pygame.draw.circle(screen, (250, 250, 250), (width/2, height/2), 180)
    s_start.draw(screen)
    pygame.display.update()   
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RETURN:
                pos = pygame.mouse.get_pos() # Mouse location when pressed
                if s_start.isOver(pos):
                    start_ticks=pygame.time.get_ticks()
                    intro = False
                    running = True
        if event.type == pygame.QUIT:
            intro=False
            pygame.quit()
            gamelog.close()
        #ESC key exits the game
        if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
            intro=False
            running = False
            pygame.quit()
            gamelog.close()
            
            
while running:
    drawGameScreen()

    for event in pygame.event.get():  # Listening to player moves
        
        #key press functions
        keyPressed()
        
        #ESC key exits the game
        if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
            running = False
            pygame.quit()
            gamelog.close()
            
        #default exit game rule
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()
            gamelog.close()
    
    while endGame:
        screen.fill((224, 224, 224))
        end_txt.draw(screen)
        end_score_txt.draw(screen)
        end_score_nr.number = str(gameStats.score)
        end_score_nr.draw(screen)
        end_restart.draw(screen)
        pygame.display.update()
        for event in pygame.event.get():
            #ESC key exits the game
            if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                running = False
                pygame.quit()
                gamelog.close()

            #default exit game rule
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()
                gamelog.close()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN:
                    gameStats.lives = 5
                    lives_nr.number = '5'
                    gameStats.score = 0
                    score_nr.number = '0'
                    gameID = random.randrange(0, 10000000, 1)
                    gameStats.probs = [1 / 7] * 7
                    endGame = False
                elif event.key == pygame.K_ESCAPE:
                    endGame = False
                    running = False
                    pygame.quit()
                    gamelog.close()
            

300 841
1526 156
1611 347
1688 907
608 163
566 434
264 260
1703 259
558 463
516 161
521 576
277 627
1536 238
372 400
1588 572
1707 847
173 803
667 188
1689 870
205 241
534 483
1173 910
1748 911
1456 880
169 665
187 267
1414 742
474 174
171 658
1686 641
